In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
from bs4 import BeautifulSoup
import pandas as pd
import time
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

# options = Options()
# options.add_argument("user-data-dir=/Users/your_user/Library/Application Support/Google/Chrome/Default")  # path to your Chrome profile
# driver = webdriver.Chrome(options=options)


# Setup Chrome options
options = Options()
options.add_argument("--start-maximized")

# Optional: Headless mode
options.add_argument("--headless")

# Start browser
driver = webdriver.Chrome(options=options)
driver.get("https://etender.cpwd.gov.in/")
time.sleep(5)

# Handle popup alert (CPWD Signer alert)
try:
    alert = driver.switch_to.alert
    print("Alert detected:", alert.text)
    alert.accept()
    print("Alert dismissed.")
    time.sleep(3)
except NoAlertPresentException:
    print("No alert found.")

# Click on "New Tenders"
driver.find_element(By.LINK_TEXT, "New Tenders").click()
time.sleep(3)

# Click on "All"
driver.find_element(By.LINK_TEXT, "All").click()
time.sleep(5)

# Grab page source after JS rendering
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Extract table
table = soup.find("table", {"id": "table"})
if table is None:
    raise Exception("Tender table not found.")

rows = table.find_all("tr")[1:21]  # Skip header row

# Extract relevant columns
data = []
for row in rows:
    cols = row.find_all("td")
    if len(cols) >= 6:
        data.append({
            "NIT/RFP NO": cols[0].text.strip(),
            "Name of Work / Subwork / Packages": cols[1].text.strip(),
            "Estimated Cost": cols[2].text.strip(),
            "Bid Submission Closing Date & Time": cols[3].text.strip(),
            "EMD Amount": cols[4].text.strip(),
            "Bid Opening Date & Time": cols[5].text.strip()
        })

# Rename columns for CSV
csv_cols = {
    "NIT/RFP NO": "ref_no",
    "Name of Work / Subwork / Packages": "title",
    "Estimated Cost": "tender_value",
    "Bid Submission Closing Date & Time": "bid_submission_end_date",
    "EMD Amount": "emd",
    "Bid Opening Date & Time": "bid_open_date"
}

df = pd.DataFrame(data)
df.rename(columns=csv_cols, inplace=True)


# Save to CSV
df.to_csv("cpwd_tenders.csv", index=False)
print("✅ Saved as cpwd_tenders.csv")


Alert detected: To access the application, please install CPWD Signer from the Downloads section and launch it with administrator privileges. For additional information, please check the CPWD Signer User Manual.
Alert dismissed.


Exception: Tender table not found.